# KAFKA PRODUCER

This Jupyter Notebook will be used as a mock producer file for our Kafka workflow. You can imagine this file as a script inside an IoT device which triggers **for example after every x seconds**. We will use the CSV file generated in the previous step and send each data record as an event.

In [ ]:
## Import Libraries
import pandas as pd
import json
import sys
from dateutil.parser import parse
from confluent_kafka import Producer
import socket

In [ ]:
def acked(err, msg):

    '''
    This function handles callback for Kafka Producer. It handles the error/success messages
    '''
    
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg.value()), str(err)))
    else:
        print("Message produced: %s" % (str(msg.value())))

The next section runs in the following order:

1. Initialise variables: CSV filepath and topic name
2. Topic: Every Kafka message should be associated to a kafka topic, it can be named anything
3. conf: Init Kafka server with IP:Port

In [ ]:
topic = "pm25_stream"
p_key = "./data/sample_multilocation.csv"

In [ ]:
### START: AVOID MAKING CHANGES ###
conf = {'bootstrap.servers': "kafka:9093", 'client.id': socket.gethostname()}
producer = Producer(conf)
### END: AVOID MAKING CHANGES ###

In [ ]:
# Read CSV using Pandas
df = pd.read_csv(p_key)

In [ ]:
# Init For Loop for number of records in CSV file
for i in range(df.shape[0]): ## df.shape returns dimension of the dataframe (rows, columns)

    result = {} ## Init Dict
    result[df.loc[i,'value']] = [df.loc[i,'lat'], df.loc[i,'lon'], str(df.loc[i,'day']), df.loc[i,'boxId']]

    '''
    Format of result JSON:

    {
        'pm25_value_1': [lat, lon, day, boxId]
    }
    '''

    # Store as JSON as Kafka supports JSON transmission as standard
    result = json.dumps(result)
    
    ## Key is optional, used to categorize messages by partition, in this case all messages get the same partition name. 
    ## Is Mostly used for scalability 

    producer.produce(topic, key=p_key, value=result, callback=acked) ## Message is stored in broker memory at this point

    ## Complete the sending of message from buffer (broker memory) and get an acknowledgement
    producer.flush()

# KAFKA CONSUMER

To now consume/processs this data you can use the next jupyter notebook: *src/step_3_event_processing.ipynb*